In [2]:
import pandas as pd
import re
from Bio import SeqIO
import numpy as np


In [78]:
#start with correct values

# only gaps of full seq need to be checked this way
hcov= 'NL63' #choose name

## enable the correct start and end based on hcov

##for 229E spike 20570..24091
# spike_start =20569
# spike_end=24091

##for HKU1 spike 22942..27012
# spike_start= 22941
# spike_end=27012

##for OC43 spike 23643..27704
# spike_start=23642
# spike_end=27704

#for NL63 spike 20472..24542
spike_start=20471
spike_end=24542

## for SARS2 spike 21563..25384
# spike_start=21562
# spike_end=25384

seq='full' #choose sequences S or full




df=pd.DataFrame()
with open('genomic_analyses//SEQ//'+str(seq)+'_SEQ//notambiguous//'+str(hcov)+'_notambiguous.fasta') as file:
    for seq_rec in SeqIO.parse(file, 'fasta'):
        seq_id= seq_rec.description
        if seq_id:
            record= {'id': seq_id, 'seq': str(seq_rec.seq.lower())}
            df=df.append(record, ignore_index=True)
print(len(df), df)

1104                                      id  \
0                           NC_045512.2   
1     hCoV-19/Argentina/INEI110208/2021   
2                           NC_045512.2   
3     hCoV-19/Argentina/INEI109388/2021   
4                           NC_045512.2   
...                                 ...   
1099     hCoV-19/Serbia/K798108-04/2022   
1100                        NC_045512.2   
1101        hCoV-19/Chile/AR-50418/2022   
1102                        NC_045512.2   
1103        hCoV-19/Chile/AR-47842/2022   

                                                    seq  
0     attaaaggtttataccttcccaggtaacaaaccaaccaactttcga...  
1     -------------------------------ccaaccaactttcga...  
2     attaaaggtttataccttcccaggtaacaaaccaaccaactttcga...  
3     --------------------------------------actttcga...  
4     attaaaggtttataccttcccaggtaacaaaccaaccaactttcga...  
...                                                 ...  
1099  -----------------------gtaacaaaccaaccaactttcga...  
1100  attaaagg

In [79]:
#cut out spike sequence from entire sequences

print(spike_start, spike_end)


result=[]
p = re.compile("-")
for i, row in df.iterrows():
    # change this depending on the hCoV you are looking at!
    if row['id']== df.id[0]:
        spike_start1= spike_start
        spike_end1= spike_end
        for m in p.finditer(str(row['seq'])):
            # print(m.start())
            if m.start() < spike_start1:
                spike_start1+=1
                spike_end1+=1
            if (m.start() > spike_start1) & (m.start() < spike_end1):
                spike_end1+=1
    spike_result= df.seq[i][spike_start1:spike_end1]
    
    result.append(spike_result)
        
df['spike_seq']=result
    
df



21562 25384


,id,seq,spike_seq
0,NC_045512.2,attaaaggtttataccttcccaggtaacaaaccaaccaactttcga...,atgtttgtttttcttgttttattgccactagtctctagtcagtgtg...
1,hCoV-19/Argentina/INEI110208/2021,-------------------------------ccaaccaactttcga...,atgtttgtttttcttgttttattgccactagtctctagtcagtgtg...
2,NC_045512.2,attaaaggtttataccttcccaggtaacaaaccaaccaactttcga...,atgtttgtttttcttgttttattgccactagtctctagtcagtgtg...
3,hCoV-19/Argentina/INEI109388/2021,--------------------------------------actttcga...,atgtttgtttttcttgttttattgccactagtctctagtcagtgtg...
4,NC_045512.2,attaaaggtttataccttcccaggtaacaaaccaaccaactttcga...,atgtttgtttttcttgttttattgccactagtctctagtcagtgtg...
...,...,...,...
1099,hCoV-19/Serbia/K798108-04/2022,-----------------------gtaacaaaccaaccaactttcga...,atgtttgtttttcttgttttattgccactagtctctagtcagtgtg...
1100,NC_045512.2,attaaaggtttataccttcccaggtaacaaaccaaccaactttcga...,atgtttgtttttcttgttttattgccactagtctctagtcagtgtg...
1101,hCoV-19/Chile/AR-50418/2022,--------------ccttcctaggtaacaaaccaaccaactttcga...,atgtttgtttttcttgttttattgccactagtttctagtcagtgtg...
1102,NC_045512.2,attaaaggtttataccttcccaggtaacaaaccaaccaactttcga...,atgtttgtttttcttgttttattgccactagtctctagtcagtgtg...


In [80]:
df['spike_seq'] = df['spike_seq'].str.replace('-','')
print(list(df['spike_seq'])[1])

atgtttgtttttcttgttttattgccactagtctctagtcagtgtgttaattttacaaacagaactcaattaccctctgcatacactaattctttcacacgtggtgtttattaccctgacaaagttttcagatcctcagttttacattcaactcaggacttgttcttacctttcttttccaatgttacttggttccatgctatacatgtctctgggaccaatggtactaagaggtttgataaccctgtcctaccatttaatgatggtgtttattttgcttccactgagaagtctaacataataagaggctggatttttggtactactttagattcgaagacccagtccctacttattgttaataacgctactaatgttgttattaaagtctgtgaatttcaattttgtaattatccatttttgggtgtttattaccacaaaaacaacaaaagttggatggaaagtgagttcagagtttattctagtgcgaataattgcacttttgaatatgtctctcagccttttcttatggaccttgaaggaaaacagggtaatttcaaaaatcttagtgaatttgtgtttaagaatattgatggttattttaaaatatattctaagcacacgcctattaatttagtgcgtgatctccctcagggtttttcggctttagaaccattggtagatttgccaataggtattaacatcactaggtttcaaactttacttgctttacatagaagttatttgactcctggtgattcttcttcaggttggacagctggtgctgcagcttattatgtgggttatcttcaacctaggacttttctattaaaatataatgaaaatggaaccattacagatgctgtagactgtgcacttgaccctctctcagaaacaaagtgtacgttgaaatccttcactgtagaaaaaggaatctatcaaacttctaactttagagtccaaccaacagaatctattgttagatttcctaatattacaa

In [81]:
#remove all even numbers (these are ref sequences)
listref=[]
for i in range(0, len(df)):
    if (i % 2 == 0) and (i!=0):
        listref.append(i)
df= df.drop(listref).reset_index(drop=True)
df

,id,seq,spike_seq
0,NC_045512.2,attaaaggtttataccttcccaggtaacaaaccaaccaactttcga...,atgtttgtttttcttgttttattgccactagtctctagtcagtgtg...
1,hCoV-19/Argentina/INEI110208/2021,-------------------------------ccaaccaactttcga...,atgtttgtttttcttgttttattgccactagtctctagtcagtgtg...
2,hCoV-19/Argentina/INEI109388/2021,--------------------------------------actttcga...,atgtttgtttttcttgttttattgccactagtctctagtcagtgtg...
3,hCoV-19/CostaRica/INC-2174-721357/2021,----aaggtttataccttcccaggtaacaaaccaaccaactttcga...,atgtttgtttttcttgttttattgccactagtctctagtcagtgtg...
4,hCoV-19/CostaRica/INC-0600/2021,------ggtttataccttcccaggtaacaaaccaaccaactttcga...,atgtttgtttttcttgttttattgccactagtctctagtcagtgtg...
...,...,...,...
548,hCoV-19/Iraq/KR-SEARCH-120460/2022,----------------------------------------------...,nnntttgtttttcttgttttattgccactagtctctagtcagtgtg...
549,hCoV-19/USA/WA-CDC-QDX36219617/2022,----------------------------------------------...,atgtttgtttttcttgttttattgccactagtctctagtcagtgtg...
550,hCoV-19/Serbia/K798108-04/2022,-----------------------gtaacaaaccaaccaactttcga...,atgtttgtttttcttgttttattgccactagtctctagtcagtgtg...
551,hCoV-19/Chile/AR-50418/2022,--------------ccttcctaggtaacaaaccaaccaactttcga...,atgtttgtttttcttgttttattgccactagtttctagtcagtgtg...


In [82]:
df['id']=df['id'].str.replace(' ', '')
df

,id,seq,spike_seq
0,NC_045512.2,attaaaggtttataccttcccaggtaacaaaccaaccaactttcga...,atgtttgtttttcttgttttattgccactagtctctagtcagtgtg...
1,hCoV-19/Argentina/INEI110208/2021,-------------------------------ccaaccaactttcga...,atgtttgtttttcttgttttattgccactagtctctagtcagtgtg...
2,hCoV-19/Argentina/INEI109388/2021,--------------------------------------actttcga...,atgtttgtttttcttgttttattgccactagtctctagtcagtgtg...
3,hCoV-19/CostaRica/INC-2174-721357/2021,----aaggtttataccttcccaggtaacaaaccaaccaactttcga...,atgtttgtttttcttgttttattgccactagtctctagtcagtgtg...
4,hCoV-19/CostaRica/INC-0600/2021,------ggtttataccttcccaggtaacaaaccaaccaactttcga...,atgtttgtttttcttgttttattgccactagtctctagtcagtgtg...
...,...,...,...
548,hCoV-19/Iraq/KR-SEARCH-120460/2022,----------------------------------------------...,nnntttgtttttcttgttttattgccactagtctctagtcagtgtg...
549,hCoV-19/USA/WA-CDC-QDX36219617/2022,----------------------------------------------...,atgtttgtttttcttgttttattgccactagtctctagtcagtgtg...
550,hCoV-19/Serbia/K798108-04/2022,-----------------------gtaacaaaccaaccaactttcga...,atgtttgtttttcttgttttattgccactagtctctagtcagtgtg...
551,hCoV-19/Chile/AR-50418/2022,--------------ccttcctaggtaacaaaccaaccaactttcga...,atgtttgtttttcttgttttattgccactagtttctagtcagtgtg...


In [83]:
def dataframetofasta(dataframe, fastaname):
        #create fasta file
        file= fastaname + ".fasta"
        f= open(file, 'a')
        #write to the file
        for i, row in dataframe.iterrows():
            lines='>'+ row['id']+'\n'+ row['spike_seq'] + "\n"
            f.write(lines)
        f.close()


#change name of the file with every run
dataframetofasta(df, 'genomic_analyses//SEQ//MSA//'+str(hcov)+'spike_MSA_'+str(seq))

